In [1]:
import numpy as np

# Classes

In [2]:
class Node:
    def __init__(self, state, children, parent, cost_from_parent, h):
        self.state = state
        self.children = list()
        self.parent = parent
        self.cost_from_parent = cost_from_parent
        self.heuristic = h
    
    def solution_dump(self, iteration):
        solution_path = list()
        total_cost = 0
        while self is not None:
            solution_path.append(self.state)
            total_cost += self.cost_from_parent
            self = self.parent

        solution_path.reverse()
        print("*** FINAL ITERATION: %d" % iteration) 
        print("*** SOLUTION = " + str(solution_path))
        print("*** TOTAL COST: %d\n" % total_cost)

In [3]:
class Graph:
    def __init__(self, graph_infos, heuristics):
        self.heuristics = heuristics
        self.graph_nodes = list(graph_infos.keys())
        self.graph_arcs = np.zeros((len(self.graph_nodes),len(self.graph_nodes)), dtype=np.int)
    
        arcs_list = list()
        for node in self.graph_nodes:
            for neighbor in graph_infos[node]:
                arcs_list.append((node,neighbor))
    
        for arc in arcs_list:
# An Highlight of the "arc" structure:
# - arc[0] contains a Node
# - arc[1] is a Tuple which contains the informations about the arc[0]'s Neighbor
    # - arc[1][0] contains a Node
    # - arc[1][1] specifies the cost to move between arc[0] and arc[1][0]
            node_idx = self.graph_nodes.index(arc[0])
            neighbor_idx = self.graph_nodes.index(arc[1][0])
            cost = arc[1][1]
            self.graph_arcs[node_idx][neighbor_idx] = cost
        
    def print_graph_description(self):
        print("\n***GRAPH DESCRIPTION***")
        for node in self.graph_nodes:
            node_idx = self.graph_nodes.index(node)
            print("%s" % node + " -> " + str(self.graph_arcs[node_idx]))

# *** *********************** ***
# *** GREEDY SEARCH FUNCTIONS ***
# *** *********************** ***
    
    def greedy_search_tree_mode(self):
        print("********************************")
        print("*** GREEDY SEARCH, TREE MODE ***")
        print("********************************\n")
        starting_state = None
# The Search begins from a Starting Node, which must be in the Graph.
        while (starting_state is None):
            print("Starting State: ", end="")
            state = input()
            if state in self.graph_nodes:
                starting_state = state
            else:
                print("This State is not in the Graph!\n")

# ALGORITHM'S EXPLANATION: this version exploits the "Tree Search" model, in which the Fringe contains all the Children of the Expanded Node
# even if they contain an "already-visited" state. 
        print("\n*** STARTING THE SEARCH... ***")
        fringe = list()

        solution_node = None

# STEP 1: We decide to expand the first Node in the Fringe, which is the Fringe's Node with the lowest value of the Heuristic Function "h".
        iterate = True
        iteration = 1
        while iterate == True:
            if iteration == 1:
                current_node = Node(state=starting_state, parent=None, cost_from_parent=0, h=self.heuristics[starting_state], children=list())
            else:
                current_node = fringe.pop(0)
            
            print("ITERATION: %d" % iteration)
            print("Expanded Node: %s" % current_node.state)

# STEP 2: For the "current_node" we execute the Goal Test: if it succeeds we have found the Solution and the Algorithm ends, otherwise we retrieve all the Neighbors.            
            current_node_idx = self.graph_nodes.index(current_node.state)
            
            if int(current_node.heuristic) == 0:
                iterate = False
                solution_node = current_node
                print("*** SOLUTION FOUND! END OF THE SEARCH ***\n")
                break

            for i in range(len(self.graph_arcs[current_node_idx])):
                cost_from_parent = self.graph_arcs[current_node_idx][i]
                if cost_from_parent != 0:
                    state = self.graph_nodes[i]
                    new_neighbor = Node(state=state, parent=current_node, cost_from_parent=cost_from_parent, h=self.heuristics[state], children=list())
                    current_node.children.append(new_neighbor)

# STEP 3: The Neighbors of the "current_node" are added to the Fringe.
# Insight: As said before, the Fringe is ordered with the "Increasing H Value" rule. The first Nodes are estimated to be closer to the Objective.
            for node in current_node.children:
                if len(fringe) == 0:
                    fringe.append(node)
                else:
                    for fringe_idx in range(len(fringe)):
                        if fringe[fringe_idx].heuristic > node.heuristic:
                            fringe.insert(fringe_idx,node)
                            break
                if node not in fringe:
                    fringe.append(node)

# The Algorithm ends when it founds the Solution or if the Fringe is empty.            
            print("Fringe: [ ", end="")
            for node in fringe:
                print(node.state + " ", end="")
            print("]")

            if len(fringe) == 0:
                iterate = False
                print("*** EMPTY FRINGE! END OF THE SEARCH ***\n")
                break

            iteration += 1
            print()

        if solution_node is not None:
            solution_node.solution_dump(iteration)
        else:
            print("*** THERE IS NOT AN OBJECTIVE NODE IN THE GRAPH! ***\n")

    def greedy_search_graph_mode(self):
        print("*********************************")
        print("*** GREEDY SEARCH, GRAPH MODE ***")
        print("*********************************\n")
        starting_state = None
# The Search begins from a Starting Node, which must be in the Graph.
        while (starting_state is None):
            print("Starting State: ", end="")
            state = input()
            if state in self.graph_nodes:
                starting_state = state
            else:
                print("This State is not in the Graph!\n")

# ALGORITHM'S EXPLANATION: this version exploits the "Graph Search" model, in which the Fringe contains only the "not-yet-expanded" Nodes. 
        print("\n*** STARTING THE SEARCH... ***")
        fringe = list()
        closed_list = list()

        solution_node = None

# STEP 1: We decide to expand the first Node in the Fringe, which is the Fringe's Node with the lowest value of the Heuristic Function "h".
        iterate = True
        iteration = 1
        while iterate == True:
            if iteration == 1:
                current_node = Node(state=starting_state, parent=None, cost_from_parent=0, h=self.heuristics[starting_state], children=list())
            else:
                current_node = fringe.pop(0)
            
            print("ITERATION: %d" % iteration)
            print("Expanded Node: %s" % current_node.state)

            if current_node.state not in closed_list:
                closed_list.append(current_node.state)

# STEP 2: For the "current_node" we execute the Goal Test: if it succeeds we have found the Solution and the Algorithm ends, otherwise we retrieve all the Neighbors.            
            current_node_idx = self.graph_nodes.index(current_node.state)
            
            if int(current_node.heuristic) == 0:
                iterate = False
                solution_node = current_node
                print("*** SOLUTION FOUND! END OF THE SEARCH ***\n")
                break

            for i in range(len(self.graph_arcs[current_node_idx])):
                cost_from_parent = self.graph_arcs[current_node_idx][i]
                if cost_from_parent != 0:
                    state = self.graph_nodes[i]
                    new_neighbor = Node(state=state, parent=current_node, cost_from_parent=cost_from_parent, h=self.heuristics[state], children=list())
                    current_node.children.append(new_neighbor)

# STEP 3: The Neighbors of the "current_node" are added to the Fringe.
# Insight: as said before...
#   - The Fringe is ordered with the "Increasing H Value" rule. The first Nodes are estimated to be closer to the Objective.
#   - The Fringe contains only the Nodes whose stats are "not-yet-visited".
            for node in current_node.children:
                if node.state not in closed_list:
                    if len(fringe) == 0:
                        fringe.append(node)
                    else:
                        for fringe_idx in range(len(fringe)):
                            if fringe[fringe_idx].heuristic > node.heuristic:
                                fringe.insert(fringe_idx,node)
                                break
                        if node not in fringe:
                            fringe.append(node)

# The Algorithm ends when it founds the Solution or if the Fringe is empty.            
            print("Fringe: [ ", end="")
            for node in fringe:
                print(node.state + " ", end="")
            print("]")

            if len(fringe) == 0:
                iterate = False
                print("*** EMPTY FRINGE! END OF THE SEARCH ***\n")
                break

            iteration += 1
            print()

        if solution_node is not None:
            solution_node.solution_dump(iteration)
        else:
            print("*** THERE IS NOT AN OBJECTIVE NODE IN THE GRAPH! ***\n")

# CODE'S TEST SECTION

In [4]:
graph_infos = dict()
graph_infos['Bus Stop'] = [('Library',2)]
graph_infos['Library'] = [('Bus Stop',2),('Student Center',2),('Car Park',5)]
graph_infos['Student Center'] = [('Library',2),('Theater',1.41),('Store',4)]
graph_infos['Theater'] = [('Student Center',1.41),('Sports Center',2)]
graph_infos['Sports Center'] = [('Theater',2),('Store',1.41)]
graph_infos['Store'] = [('Student Center',4),('Sports Center',1.41),('Car Park',5),('Canteen',3)]
graph_infos['Car Park'] = [('Library',5),('Store',5),('Maths Building',4.24)]
graph_infos['Maths Building'] = [('Car Park',4.24),('Canteen',2)]
graph_infos['Canteen'] = [('AI Lab',1),('Maths Building',2),('Store',3)]
graph_infos['AI Lab'] = [('Canteen',1)]

h = dict() # Distanza in linea d'aria da "AI Lab" (Obbiettivo)
h['Bus Stop'] = 8.94
h['Library'] = 8.25
h['Student Center'] = 8.0
h['Theater'] = 7.07
h['Sports Center'] = 5.1
h['Store'] = 4.0
h['Car Park'] = 6.4
h['Maths Building'] = 2.24
h['Canteen'] = 1.0
h['AI Lab'] = 0.0

In [5]:
graph = Graph(graph_infos, h)

C:\Users\Riccardo De Cesaris\AppData\Local\Temp\ipykernel_12692\641404139.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.graph_arcs = np.zeros((len(self.graph_nodes),len(self.graph_nodes)), dtype=np.int)


## Greedy Search, Tree Mode

In [6]:
graph.greedy_search_tree_mode()

********************************
*** GREEDY SEARCH, TREE MODE ***
********************************

Starting State: Bus Stop

*** STARTING THE SEARCH... ***
ITERATION: 1
Expanded Node: Bus Stop
Fringe: [ Library ]

ITERATION: 2
Expanded Node: Library
Fringe: [ Car Park Student Center Bus Stop ]

ITERATION: 3
Expanded Node: Car Park
Fringe: [ Maths Building Store Student Center Library Bus Stop ]

ITERATION: 4
Expanded Node: Maths Building
Fringe: [ Canteen Store Car Park Student Center Library Bus Stop ]

ITERATION: 5
Expanded Node: Canteen
Fringe: [ AI Lab Maths Building Store Store Car Park Student Center Library Bus Stop ]

ITERATION: 6
Expanded Node: AI Lab
*** SOLUTION FOUND! END OF THE SEARCH ***

*** FINAL ITERATION: 6
*** SOLUTION = ['Bus Stop', 'Library', 'Car Park', 'Maths Building', 'Canteen', 'AI Lab']
*** TOTAL COST: 14



## Greedy Search, Graph Mode

In [7]:
graph.greedy_search_graph_mode()

*********************************
*** GREEDY SEARCH, GRAPH MODE ***
*********************************

Starting State: Bus Stop

*** STARTING THE SEARCH... ***
ITERATION: 1
Expanded Node: Bus Stop
Fringe: [ Library ]

ITERATION: 2
Expanded Node: Library
Fringe: [ Car Park Student Center ]

ITERATION: 3
Expanded Node: Car Park
Fringe: [ Maths Building Store Student Center ]

ITERATION: 4
Expanded Node: Maths Building
Fringe: [ Canteen Store Student Center ]

ITERATION: 5
Expanded Node: Canteen
Fringe: [ AI Lab Store Store Student Center ]

ITERATION: 6
Expanded Node: AI Lab
*** SOLUTION FOUND! END OF THE SEARCH ***

*** FINAL ITERATION: 6
*** SOLUTION = ['Bus Stop', 'Library', 'Car Park', 'Maths Building', 'Canteen', 'AI Lab']
*** TOTAL COST: 14

